In [1]:
import pandas as pd;
car_eff = pd.read_csv('car_dataset_with_imputer.csv')

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(car_eff, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_test.shape)

(318, 9)
(80, 9)


In [3]:
Y_train = X_train['mpg']
X_train.drop('mpg', inplace=True, axis=1)

/home/dee/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [4]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train_scaled)
X_test_scaled = sc.fit_transform(X_test)
X_train_scaled

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[-1.22370421,  1.49526939,  1.22961301, ..., -1.13752513,
        -0.73301171,  1.23868518],
       [-1.30308591, -0.85285735, -0.92367663, ..., -1.41177304,
         0.5068698 , -1.16990273],
       [ 1.00780372, -0.85285735, -0.92367663, ...,  1.05645814,
         0.5068698 , -0.68818515],
       ...,
       [-0.75623417, -0.85285735, -1.206235  , ..., -0.86327722,
         0.5068698 , -1.4642857 ],
       [-1.37364743,  0.32120602,  0.56706235, ..., -1.41177304,
        -0.73301171, -0.09941922],
       [-0.27112375, -0.85285735, -0.99188037, ..., -0.31478141,
         0.5068698 , -0.87551976]])

In [5]:
#Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

In [6]:
lin_reg.fit(X_train_scaled, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [7]:
some_data = X_train.iloc[:5]
some_label = Y_train.iloc[:5]
some_data = sc.fit_transform(some_data)
print('Predictions:', lin_reg.predict(some_data))
print('Label:', list(some_label))

Predictions: [10.02747117 24.96257595 27.85417168 26.47515367 28.55484137]
Label: [15.0, 27.0, 34.3, 26.0, 31.6]


/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [8]:
import numpy as np
from sklearn.metrics import mean_squared_error
lin_predictions = lin_reg.predict(X_train_scaled)
lin_mse = mean_squared_error(Y_train, lin_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

3.296616879634875

# Validation

In [9]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_reg, X_train_scaled, Y_train, scoring="neg_mean_squared_error", cv=20)
lin_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(lin_rmse_scores)

Scores: [3.59138919 2.25625446 4.15973519 4.1213422  3.52167219 3.15793335
 3.96898087 3.98719947 3.92662417 3.37631322 3.70034485 2.91643369
 3.12098894 3.92042023 1.86730029 2.02286037 4.46019014 3.03236172
 2.70222334 2.26662144]
Mean: 3.3038594662425225
Standard deviation 0.7492096730341077


# Hyperparameter Tuning

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'copy_X': [True, False], 'fit_intercept':[True], 'n_jobs':[None, 2, 4, 6, 8, 10], 'normalize':[False, True]}
]
grid_search = GridSearchCV(lin_reg, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'copy_X': [True, False], 'fit_intercept': [True], 'n_jobs': [None, 2, 4, 6, 8, 10], 'normalize': [False, True]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [11]:
grid_search.best_estimator_

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [12]:
grid_search.best_params_

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True}

In [13]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(grid_search.best_estimator_, X_train_scaled, Y_train, scoring="neg_mean_squared_error", cv=20)
lin_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(lin_rmse_scores)

Scores: [3.59138919 2.25625446 4.15973519 4.1213422  3.52167219 3.15793335
 3.96898087 3.98719947 3.92662417 3.37631322 3.70034485 2.91643369
 3.12098894 3.92042023 1.86730029 2.02286037 4.46019014 3.03236172
 2.70222334 2.26662144]
Mean: 3.3038594662425225
Standard deviation 0.7492096730341081


In [14]:
from sklearn.metrics import mean_squared_error
lin_predictions = grid_search.best_estimator_.predict(X_train_scaled)
lin_mse = mean_squared_error(Y_train, lin_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

3.296616879634875

Linear Regression is highly inaccurate